## Question - 2 : Fuzzy inference system for Restaurant Tipping Problem

**Note: Kindly run it on google colab or to run on local, comment the below cell and install scikit-fuzzy**

In [1]:
!pip install -U scikit-fuzzy

Requirement already up-to-date: scikit-fuzzy in /usr/local/lib/python3.7/dist-packages (0.4.2)


In [2]:
import numpy as np
import skfuzzy as fuzz

## Mamdani Method

In [3]:
def findTipUsingMamdani(service_score, food_score):

  x_service = np.arange(0, 10.01, 0.5)
  x_food = np.arange(0, 10.01, 0.5)
  x_tip = np.arange(0, 25.01, 1.0)

  service_low = fuzz.trimf(x_service, [0, 0, 5])
  service_middle = fuzz.trimf(x_service, [0, 5, 10])
  service_high = fuzz.trimf(x_service, [5, 10, 10])

  food_low = fuzz.zmf(x_food, 0, 5)
  food_middle = fuzz.pimf(x_food, 0, 4, 5, 10)
  food_high = fuzz.smf(x_food, 5, 10)

  tip_low = fuzz.trimf(x_tip, [0, 0, 13])
  tip_middle = fuzz.trimf(x_tip, [0, 13, 25])
  tip_high = fuzz.trimf(x_tip, [13, 25, 25])

  service_low_degree = fuzz.interp_membership(
      x_service, service_low, service_score)
  service_middle_degree = fuzz.interp_membership(
      x_service, service_middle, service_score)
  service_high_degree = fuzz.interp_membership(
      x_service, service_high, service_score)

  food_low_degree = fuzz.interp_membership(x_food, food_low, food_score)
  food_middle_degree = fuzz.interp_membership(x_food, food_middle, food_score)
  food_high_degree = fuzz.interp_membership(x_food, food_high, food_score)

  low_degree = np.fmax(service_low_degree, food_low_degree)
  middle_degree = service_middle_degree
  high_degree = np.fmax(service_high_degree, food_high_degree)

  activation_low = np.fmin(low_degree, tip_low)
  activation_middle = np.fmin(middle_degree, tip_middle)
  activation_high = np.fmin(high_degree, tip_high)

  aggregated = np.fmax(
      activation_low,
      np.fmax(activation_middle, activation_high))

  tip_value = fuzz.defuzz(x_tip, aggregated, 'centroid')
  return tip_value

In [4]:
print( findTipUsingMamdani( service_score = 9.5, food_score = 4.0) )

18.389729481889038


## Sugeno Method

In [5]:
def findTipUsingSugeno( service_score, food_score):
  x_service = np.arange(0, 10.01, 0.5)
  x_food = np.arange(0, 10.01, 0.5)
  x_tip = np.arange(0, 25.01, 1.0)

  service_low = fuzz.trimf(x_service, [0, 0, 5])
  service_middle = fuzz.trimf(x_service, [0, 5, 10])
  service_high = fuzz.trimf(x_service, [5, 10, 10])

  food_low = fuzz.zmf(x_food, 0, 5)
  food_middle = fuzz.pimf(x_food, 0, 4, 5, 10)
  food_high = fuzz.smf(x_food, 5, 10)

  f, s = np.meshgrid(x_food, x_service, indexing="ij")
  tip_low_grid = 5.0 + s * 0.2 + f * 0.2
  tip_middle_grid = 5.0 + s * 0.5 + f * 0.5
  tip_high_grid = 5.0 + s + f

  service_low_degree = fuzz.interp_membership(
      x_service, service_low, service_score)
  service_middle_degree = fuzz.interp_membership(
      x_service, service_middle, service_score)
  service_high_degree = fuzz.interp_membership(
      x_service, service_high, service_score)

  food_low_degree = fuzz.interp_membership(x_food, food_low, food_score)
  food_middle_degree = fuzz.interp_membership(x_food, food_middle, food_score)
  food_high_degree = fuzz.interp_membership(x_food, food_high, food_score)

  low_degree = np.fmax(service_low_degree, food_low_degree)
  middle_degree = service_middle_degree
  high_degree = np.fmax(service_high_degree, food_high_degree)

  w1 = low_degree
  w2 = middle_degree
  w3 = high_degree

  z1 = 5.0 + 0.2 * food_score + 0.2 * service_score
  z2 = 5.0 + 0.5 * food_score + 0.5 * service_score
  z3 = 5 + 1.0 * food_score + 1.0 * service_score
  tip_value = (w1 * z1 + w2 * z2 + w3 * z3) / (w1 + w2 + w3)
  return tip_value


In [6]:
print( findTipUsingSugeno( service_score = 9.5, food_score = 4.0) )

17.075000000000003


References:
1. https://functionbay.com/documentation/onlinehelp/default.htm#!Documents/introductiontothefisfileformat.htm
2. https://github.com/caigen/scikit-fuzzy-examples
3. https://thesocialcomment.com/project/full/Fuzzy-Control-Systems--The-Tipping-Problem?pid=5e0b52a22a37d20505da2a49